In [193]:
import os, sys
import random 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
%matplotlib inline

In [194]:
# load all the .csv files within one folder
# return -> dict: {key=csv filename, value=pd.DataFrame}
def load_all_files(path):
    d = {}
    for root, dirs, files in os.walk(path):
        for file in files:
            d[file.split(".")[0]] = pd.read_csv(os.path.join(root, file), index_col=0)
    return d

In [195]:
data_dict = load_all_files("./preprocessed_data_Barry")
data_dict_t = load_all_files("./preprocessed_data_Tya")

In [178]:
data_dict_t.keys()

dict_keys(['new_cases', 'new_cases_per_million', 'new_deaths', 'new_deaths_per_million', 'new_tests', 'positive_rate', 'stringency_index', 'total_cases', 'total_deaths', 'total_deaths_per_million', 'total_tests'])

In [179]:
for k, df in data_dict_t.items():
    data_dict_t[k] = df.iloc[:-1,:].copy()

In [197]:
data_dict["new_cases"] = data_dict["new_cases"].iloc[:-1, 1:].copy()

In [198]:
# select features & country data to be included into the model
selected_features = ['C1_School_closing', 'C2_Workplace_closing', 'C3_Cancel_public_events', 'C4_Restrictions_on_gatherings',
                     'C5_Close_public_transport', 'C6_Stay_at_home_requirements', 'C7_Restrictions_on_internal_movement', 
                     'C8_International_travel_controls', 'E1_Income_support', 'E2_Debt_contract_relief', 
                     'E3_Fiscal_measures', 'E4_International_support', 'H1_Public_information_campaigns', 'H2_Testing_policy', 'H3_Contact_tracing',
                     'H4_Emergency_investment_in_healthcare', 'H5_Investment_in_vaccines', 'H6_Facial_Coverings', 'I_ContainmentHealthIndex', 
                     'I_GovernmentResponseIndex', 'I_StringencyIndex', 'new_cases']

selected_features_t = ['new_cases_per_million', 'new_deaths', 'new_deaths_per_million', 
                       'new_tests', 'positive_rate', 'stringency_index', 'total_deaths', 
                       'total_deaths_per_million', 'total_tests', 'new_cases']

# selected_country = ["ARG"]
# selected_country_t = ["JPN"]
selected_country = list(data_dict['C1_School_closing'].index)
selected_country_t = list(data_dict_t['total_cases'].index)

In [199]:
# create a date * feature sample dataframe
dates = data_dict["ConfirmedCases"].columns[1:]
sample_df = pd.DataFrame(columns=selected_features, index=dates, dtype=float)

In [200]:
dates_t = data_dict_t["total_cases"].columns[1:]
sample_df_t = pd.DataFrame(columns=selected_features_t, index=dates, dtype=float)

In [201]:
sample_df_t

,new_cases_per_million,new_deaths,new_deaths_per_million,new_tests,positive_rate,stringency_index,total_deaths,total_deaths_per_million,total_tests,new_cases
2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-11-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [202]:
def do_scale(df, threshold=10, factor=100):
    for col in df.columns:
        if df[col].mean() < threshold:
            df[col] = df[col] * factor * random.randint(1,11)
    return df

In [203]:
def insert_yesterday(df):
    yesterday = [0]
    yesterday.extend(list(df["new_cases"]))
    df["YesterdayConfirmed"] =yesterday[:-1]
    return df

In [204]:
def do_regression(country, concat_df, standarize=True, scale=False, verbose=False):
    # insert corresponding feature data from dataframes
    for feature in selected_features:
        concat_df[feature] = data_dict[feature].loc[country]
            
     # clear out invalid tail data       
    concat_df = concat_df.iloc[:-25,:].copy()
    
    # insert "YesterdayConfirmed" feature
    concat_df = insert_yesterday(concat_df).copy()
    
    # replace NaN with 0
    concat_df = concat_df.fillna(0)
    if scale:
        concat_df = do_scale(concat_df, 10, 10000).copy()
    
    # create train&test data
    X = np.array(concat_df[selected_features[:-1]+ ["YesterdayConfirmed"]])
    y = np.array(concat_df[selected_features[-1]]).reshape(-1,1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)
    
    # whether to scale the data, True by default
    if standarize:
        scaler_x = StandardScaler()
        X_train = scaler_x.fit_transform(X_train)
        X_test  = scaler_x.transform(X_test)
        X_train = np.hstack((np.ones(X_train.shape[0]).reshape(-1,1), X_train))
        X_test = np.hstack((np.ones(X_test.shape[0]).reshape(-1,1), X_test))

        scaler_y = StandardScaler()
        y_train = scaler_y.fit_transform(y_train)
        y_test  = scaler_y.transform(y_test)
    
    # create linear regression model & train
#     regr = linear_model.LinearRegression()
    regr = Ridge(alpha=1.5)
#     regr = Lasso(alpha=0.001)
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    
    if verbose:
        # print out statistical data
        print("--------------------------------------------------")
        print("Country:", country)
        print('Coefficients: \n')
    #     print(regr.coef_)
        for i in range(0, len(regr.coef_[0]) -1):
            print("Feature: %s" % selected_features[i])
            print("Weight: %.6f" % regr.coef_[0][i + 1])
    #         print()
        print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
        print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))

        plt.semilogy(y_pred, label = "y_pred", color="red");
        plt.semilogy(y_test, label = "y_test", color="blue");

        plt.show()
    return regr.coef_[0][1:]

In [205]:
def do_regression_t(country, concat_df, standarize=True, scale=False, verbose=False):
    # insert corresponding feature data from dataframes
    for feature in selected_features_t:
        concat_df[feature] = data_dict_t[feature].loc[country]
            
     # clear out invalid tail data       
    concat_df = concat_df.iloc[:-50,:].copy()
    
    # insert "YesterdayConfirmed" feature
    concat_df = insert_yesterday(concat_df).copy()
    
    # replace NaN with 0
    concat_df = concat_df.fillna(0)
    if scale:
        concat_df = do_scale(concat_df, 10, 10000).copy()
    
    # create train&test data
    X = np.array(concat_df[selected_features_t[:-1]+ ["YesterdayConfirmed"]])
    y = np.array(concat_df[selected_features_t[-1]]).reshape(-1,1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)
    
    # whether to scale the data, True by default
    if standarize:
        scaler_x = StandardScaler()
        X_train = scaler_x.fit_transform(X_train)
        X_test  = scaler_x.transform(X_test)
        X_train = np.hstack((np.ones(X_train.shape[0]).reshape(-1,1), X_train))
        X_test = np.hstack((np.ones(X_test.shape[0]).reshape(-1,1), X_test))

        scaler_y = StandardScaler()
        y_train = scaler_y.fit_transform(y_train)
        y_test  = scaler_y.transform(y_test)
    
    # create linear regression model & train
#     regr = linear_model.LinearRegression()
    regr = Ridge(alpha=1.5)
#     regr = Lasso(alpha=0.001)
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    
    if verbose:
        # print out statistical data
        print("--------------------------------------------------")
        print("Country:", country)
        print('Coefficients: \n')
    #     print(regr.coef_)
        for i in range(0, len(regr.coef_[0]) -1):
            print("Feature: %s" % selected_features_t[i])
            print("Weight: %.6f" % regr.coef_[0][i + 1])
    #         print()
        print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
        print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))

        plt.semilogy(y_pred, label = "y_pred", color="red");
        plt.semilogy(y_test, label = "y_test", color="blue");

        plt.show()
    return regr.coef_[0][1:]

In [208]:
# prepare statistical dict for coefs
coefs = {}
for k in data_dict.keys():
    coefs[k] = []

In [209]:
idx = 0
verbose=False
for country in selected_country:
    print("Calculating:", country, idx)
    try:
        c = do_regression(country, sample_df, verbose=verbose)
    except KeyError:
        continue
    if not verbose:
        if (max(c) >= 5 or min(c) <= -5):
            print("WARNING:", country, c)
        else:
            for k in range(len(c)):
                coefs[selected_features[k]].append(c[k])
        idx += 1

Calculating: ABW 0
Calculating: AFG 1
Calculating: AGO 2
Calculating: ALB 3
Calculating: AND 4
Calculating: ARE 5
Calculating: ARG 6
Calculating: AUS 7
Calculating: AUT 8
Calculating: AZE 9
Calculating: BDI 10
Calculating: BEL 11
Calculating: BEN 12
Calculating: BFA 13
Calculating: BGD 14
Calculating: BGR 15
Calculating: BHR 16
Calculating: BHS 17
Calculating: BIH 18
Calculating: BLR 19
Calculating: BLZ 20
Calculating: BMU 21
Calculating: BOL 22
Calculating: BRA 23
Calculating: BRB 24
Calculating: BRN 25
Calculating: BTN 26
Calculating: BWA 27
Calculating: CAF 28
Calculating: CAN 29
Calculating: CHE 30
Calculating: CHL 31
Calculating: CHN 32
Calculating: CIV 33
Calculating: CMR 34
Calculating: COD 35
Calculating: COG 36
Calculating: COL 37
Calculating: COM 38
Calculating: CPV 39
Calculating: CRI 40
Calculating: CUB 41
Calculating: CYP 42
Calculating: CZE 43
Calculating: DEU 44
Calculating: DJI 45
Calculating: DMA 46
Calculating: DNK 47
Calculating: DOM 48
Calculating: DZA 49
Calculatin

In [211]:
for f in selected_features:
    print(f, round(np.array(coefs[f]).mean(), 5))

C1_School_closing 0.00295
C2_Workplace_closing 0.05707
C3_Cancel_public_events 0.02533
C4_Restrictions_on_gatherings 0.00476
C5_Close_public_transport 0.00355
C6_Stay_at_home_requirements 0.02335
C7_Restrictions_on_internal_movement 0.02492
C8_International_travel_controls 0.02051
E1_Income_support 0.02933
E2_Debt_contract_relief 0.02515
E3_Fiscal_measures -0.00796
E4_International_support 0.00205
H1_Public_information_campaigns 0.01166
H2_Testing_policy -0.00354
H3_Contact_tracing -0.00679
H4_Emergency_investment_in_healthcare 0.0095
H5_Investment_in_vaccines 0.00068
H6_Facial_Coverings 0.09785
I_ContainmentHealthIndex -0.00616
I_GovernmentResponseIndex 0.00598
I_StringencyIndex -0.01395
new_cases 0.31073
